In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
!nvidia-smi

Fri Apr 26 08:12:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import auth
auth.authenticate_user()


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')

In [0]:
train,test = train_test_split(df,random_state = 100)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}
labels_cat1 = {j:i for i,j in enumerate(train.cat1.unique())}
labels_cat2 = {j:i for i,j in enumerate(train.cat2.unique())}
labels_cat3 = {j:i for i,j in enumerate(train.cat3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)
Y_train_cat1 = train.apply(lambda i : labels_cat1[i.cat1], axis=1)
Y_train_cat2 = train.apply(lambda i : labels_cat2[i.cat2], axis=1)
Y_train_cat3 = train.apply(lambda i : labels_cat3[i.cat3], axis=1)


In [0]:
df.columns

Index(['Unnamed: 0', 'archive_by_user', 'brand', 'cat1', 'cat2', 'cat3',
       'city', 'created_at', 'desc', 'id', 'image_count', 'mileage',
       'platform', 'price', 'title', 'type', 'year', 'cat1_2', 'cat1_2_3'],
      dtype='object')

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)
Y_test_cat1 = test.apply(lambda i : labels_cat1[i.cat1], axis=1)
Y_test_cat2 = test.apply(lambda i : labels_cat2[i.cat2], axis=1)
Y_test_cat3 = test.apply(lambda i : labels_cat3[i.cat3], axis=1)


In [0]:
MAX_NB_WORDS = 300000

texts_train =X_train.astype(str)
texts_test = X_test.astype(str)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 438592 unique tokens.


In [0]:
index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [0]:
" ".join([index_to_word[i] for i in sequences[0]])

'پیش دستی و پلوخوری خارجی باسلام یه دست پیش دستی و یه دست پلوخوری که هرکدام مبلغش بیست و پنج هزارتومان میباشد با تخفیف باتشکر qom sunday 08am'

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
MAX_SEQUENCE_LENGTH = 200

x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)


Shape of data tensor: (710726, 250)
Shape of data test tensor: (236909, 250)


In [0]:


Y_train = to_categorical(np.asarray(Y_train))
Y_train_cat1 = to_categorical(np.asarray(Y_train_cat1))
Y_train_cat2 = to_categorical(np.asarray(Y_train_cat2))
Y_train_cat3 = to_categorical(np.asarray(Y_train_cat3))

Y_test = to_categorical(np.asarray(Y_test))
Y_test_cat1 = to_categorical(np.asarray(Y_test_cat1))
Y_test_cat2 = to_categorical(np.asarray(Y_test_cat2))
Y_test_cat3 = to_categorical(np.asarray(Y_test_cat3))

print('Shape of label tensor:', Y_train_cat1.shape)
print('Shape of label tensor:', Y_train_cat2.shape)
print('Shape of label tensor:', Y_train_cat3.shape)

print('Shape of label tensor:', Y_test_cat1.shape)
print('Shape of label tensor:', Y_test_cat2.shape)
print('Shape of label tensor:', Y_test_cat3.shape)




Shape of label tensor: (710726, 6)
Shape of label tensor: (710726, 28)
Shape of label tensor: (710726, 67)
Shape of label tensor: (236909, 6)
Shape of label tensor: (236909, 28)
Shape of label tensor: (236909, 67)


In [0]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
N_CLASSES_Cat1 = 6
N_CLASSES_Cat2 = 28
N_CLASSES_Cat3 = 67


sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

x = LSTM(128, recurrent_dropout=0.2)(embedded_sequences)
predictions_cat1 = Dense(N_CLASSES_Cat1,name='cat1', activation='softmax')(x)
predictions_cat2 = Dense(N_CLASSES_Cat2,name='cat2', activation='softmax')(x)
predictions_cat3 = Dense(N_CLASSES_Cat3, name='cat3',activation='softmax')(x)


model = Model(inputs=sequence_input, outputs=[predictions_cat1,predictions_cat2,predictions_cat3])
loss_funcs = {
        "cat1": "categorical_crossentropy",
        "cat2": "categorical_crossentropy",
        "cat3": "categorical_crossentropy"
}
metrics = {
        "cat1": ['acc'],
        "cat2": ['acc'],
        "cat3": ['acc']  


}
y_trains = {
        "cat1": Y_train_cat1,
        "cat2": Y_train_cat2,
        "cat3": Y_train_cat3
}
y_valids = {
        "cat1": Y_test_cat1,
        "cat2": Y_test_cat2,
        "cat3": Y_test_cat3
}
model.compile(optimizer='adam',loss=loss_funcs, metrics=metrics)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 100)     3000000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          117248      embedding_1[0][0]                
__________________________________________________________________________________________________
cat1 (Dense)         

In [0]:
model.fit(x_train, y_trains, 
                        batch_size= 128,
                        epochs=6,
                        validation_data=(x_test,y_valids),
                        shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 710726 samples, validate on 236909 samples
Epoch 1/6
710726/710726 [==============================] - 3190s 4ms/step - loss: 1.4933 - cat1_loss: 0.2603 - cat2_loss: 0.5012 - cat3_loss: 0.7318 - cat1_acc: 0.9139 - cat2_acc: 0.8584 - cat3_acc: 0.8012 - val_loss: 0.7582 - val_cat1_loss: 0.1505 - val_cat2_loss: 0.2518 - val_cat3_loss: 0.3559 - val_cat1_acc: 0.9510 - val_cat2_acc: 0.9266 - val_cat3_acc: 0.8977
Epoch 2/6
710726/710726 [==============================] - 3225s 5ms/step - loss: 0.6543 - cat1_loss: 0.1301 - cat2_loss: 0.2158 - cat3_loss: 0.3083 - cat1_acc: 0.9572 - cat2_acc: 0.9357 - cat3_acc: 0.9093 - val_loss: 0.6739 - val_cat1_loss: 0.1373 - val_cat2_loss: 0.2253 - val_cat3_loss: 0.3112 - val_cat1_acc: 0.9549 - val_cat2_acc: 0.9330 - val_cat3_acc: 0.9072
Epoch 3/6
435968/710726 [=================>............] - ETA: 18:12 - loss: 0.5352 - cat1_loss: 0.1068 - cat2_loss: 0.1740 - cat3_loss: 0.2544 - cat1_acc: 0.9645 - ca

In [0]:
[output_test_cat1,output_test_cat2,output_test_cat3] = model.predict(x_test)


In [0]:
Y_train_cat2

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
from sklearn.metrics import classification_report

output_test_cat1 = np.argmax(output_test_cat1,axis=1)
output_test_cat2 = np.argmax(output_test_cat2,axis=1)
output_test_cat3 = np.argmax(output_test_cat3,axis=1)

Y_test_cat1 = np.argmax(Y_test_cat1)
Y_test_cat2 = np.argmax(Y_test_cat2)
Y_test_cat3 = np.argmax(Y_test_cat3)

print(classification_report(Y_test_cat1, output_test_cat1))
print(classification_report(Y_test_cat2, output_test_cat2))
print(classification_report(Y_test_cat3, output_test_cat3))

NameError: ignored

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 150

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

In [0]:
[output_tst_cat1,output_tst_cat2,output_tst_cat3] = model.predict(x_tst)


In [0]:
pred_cat1=np.argmax(output_tst_cat1,axis=1)
pred_cat2=np.argmax(output_tst_cat2,axis=1)
pred_cat3=np.argmax(output_tst_cat3,axis=1)

In [0]:
tst['cat1']=pred_cat1
tst['cat2']=pred_cat2
tst['cat3']=pred_cat3


In [0]:
merged_cat1 = {}
for key, value in labels_cat1.items():
    merged_cat1[value] = key

merged_cat2 = {}
for key, value in labels_cat2.items():
    merged_cat2[value] = key

merged_cat3 = {}
for key, value in labels_cat3.items():
    merged_cat3[value] = key



In [0]:
pred_cat1.shape


In [0]:
tst['cat1'] = tst['cat1'].map(merged_cat1)
tst['cat2'] = tst['cat2'].map(merged_cat2)
tst['cat3'] = tst['cat3'].map(merged_cat3)

In [0]:
tst

In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title'],axis=1)


In [0]:
tst.replace([NaN], '', inplace=True)

In [0]:
tst

In [0]:
tst.to_csv("tst_deep.csv")

In [0]:
model.sa